Практическое задание 4
КМБО-06-20, Бундуки Владислав, Вариант 2

Устанавливаем необходимые библиотеки:

In [ ]:
!pip install pandas
!pip install sklearn

In [87]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
data = pd.read_csv('aug_test.csv', index_col='city')

#исключим столбец с уникальными значениями: enrollee_id	
data = data[data.columns.drop(['enrollee_id'])]
data.head()

,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
city,,,,,,,,,,,
city_41,0.827,Male,Has relevent experience,Full time course,Graduate,STEM,9,<10,NaN,1,21
city_103,0.920,Female,Has relevent experience,no_enrollment,Graduate,STEM,5,NaN,Pvt Ltd,1,98
city_21,0.624,Male,No relevent experience,no_enrollment,High School,NaN,<1,NaN,Pvt Ltd,never,15
city_13,0.827,Male,Has relevent experience,no_enrollment,Masters,STEM,11,10/49,Pvt Ltd,1,39
city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,10000+,Pvt Ltd,>4,72


Описание столбцов:
enrollee_id :  уникальный идентификатор кандидата

city: Код города

city_ development _index : индекс развития города (в масштабе)

gender: Пол кандидата

relevent_experience: Соответствующий опыт кандидата

enrolled_university: Тип зачисленных университетских курсов, если таковые имеются.

education_level: Уровень образования кандидата

major_discipline :Обучение основной дисциплине кандидата

experience: Кандидатский общий стаж в годах

company_size: Количество сотрудников в компании текущего работодателя

company_type : Тип текущего работодателя

lastnewjob: разница в годах между предыдущей работой и текущей работой

training_hours: завершенные часы обучения

target: 0 - Не ищу смены работы, 1 - Ищу смену работы

In [88]:
#Выводим информацию о датасете
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2129 entries, city_41 to city_102
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city_development_index  2129 non-null   float64
 1   gender                  1621 non-null   object 
 2   relevent_experience     2129 non-null   object 
 3   enrolled_university     2098 non-null   object 
 4   education_level         2077 non-null   object 
 5   major_discipline        1817 non-null   object 
 6   experience              2124 non-null   object 
 7   company_size            1507 non-null   object 
 8   company_type            1495 non-null   object 
 9   last_new_job            2089 non-null   object 
 10  training_hours          2129 non-null   int64  
dtypes: float64(1), int64(1), object(9)
memory usage: 199.6+ KB


In [89]:
#Посмотрим на статистики численных данных:
data.describe()

,city_development_index,training_hours
count,2129.000000,2129.000000
mean,0.824984,64.983091
std,0.125074,60.238660
min,0.448000,1.000000
25%,0.698000,23.000000
50%,0.903000,47.000000
75%,0.920000,86.000000
max,0.949000,334.000000


In [90]:
#Кодируем столбец из категориального признака
data['gender'] = np.where(data['gender'] == 'Male', 1, 0)
data['education_level'] = np.where(data['education_level'] == 'Masters', 0, 1)
data['relevent_experience'] = np.where(data['relevent_experience'] == 'No relevent experience',0,1)

In [91]:
data.gender.head()

city
city_41     1
city_103    0
city_21     1
city_13     1
city_103    1
Name: gender, dtype: int32

In [92]:
data['experience'].value_counts()

>20    383
5      163
3      154
4      145
6      130
2      128
7      116
9      113
10      96
11      86
8       82
<1      74
16      68
15      59
1       56
14      55
13      54
12      52
17      36
19      29
18      26
20      19
Name: experience, dtype: int64

In [93]:
enrolled_university = pd.get_dummies(data['enrolled_university']) 
data = pd.concat((data, enrolled_university), axis=1)

last_new_job = pd.get_dummies(data['last_new_job']) 
data = pd.concat((data, last_new_job), axis=1)

major_discipline = pd.get_dummies(data['major_discipline']) 
data = pd.concat((data, major_discipline), axis=1)

company_type = pd.get_dummies(data['company_type']) 
data = pd.concat((data, company_type), axis=1)

company_size = pd.get_dummies(data['company_size']) 
data = pd.concat((data, company_size), axis=1)

data.loc[data.experience == '<1' , 'experience'] = 0
data.loc[data.experience == 'NaN' , 'experience'] = 0
data.loc[data.experience == '>20' , 'experience'] = 21

data.loc[data.last_new_job == 'never' , 'last_new_job'] = 0
data.loc[data.last_new_job == 'NaN' , 'last_new_job'] = 0
data.loc[data.last_new_job == '>4' , 'last_new_job'] = 5

data.loc[data.gender == 'NaN' , 'gender'] = 0

data.head()

,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,...,Public Sector,Pvt Ltd,10/49,100-500,1000-4999,10000+,50-99,500-999,5000-9999,<10
city,,,,,,,,,,,,,,,,,,,,,
city_41,0.827,1,1,Full time course,1,STEM,9,<10,NaN,1,...,0,0,0,0,0,0,0,0,0,1
city_103,0.920,0,1,no_enrollment,1,STEM,5,NaN,Pvt Ltd,1,...,0,1,0,0,0,0,0,0,0,0
city_21,0.624,1,0,no_enrollment,1,NaN,0,NaN,Pvt Ltd,0,...,0,1,0,0,0,0,0,0,0,0
city_13,0.827,1,1,no_enrollment,0,STEM,11,10/49,Pvt Ltd,1,...,0,1,1,0,0,0,0,0,0,0
city_103,0.920,1,1,no_enrollment,1,STEM,21,10000+,Pvt Ltd,5,...,0,1,0,0,0,1,0,0,0,0


In [94]:
data

,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,...,Public Sector,Pvt Ltd,10/49,100-500,1000-4999,10000+,50-99,500-999,5000-9999,<10
city,,,,,,,,,,,,,,,,,,,,,
city_41,0.827,1,1,Full time course,1,STEM,9,<10,NaN,1,...,0,0,0,0,0,0,0,0,0,1
city_103,0.920,0,1,no_enrollment,1,STEM,5,NaN,Pvt Ltd,1,...,0,1,0,0,0,0,0,0,0,0
city_21,0.624,1,0,no_enrollment,1,NaN,0,NaN,Pvt Ltd,0,...,0,1,0,0,0,0,0,0,0,0
city_13,0.827,1,1,no_enrollment,0,STEM,11,10/49,Pvt Ltd,1,...,0,1,1,0,0,0,0,0,0,0
city_103,0.920,1,1,no_enrollment,1,STEM,21,10000+,Pvt Ltd,5,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
city_103,0.920,1,0,no_enrollment,1,Humanities,16,NaN,Public Sector,4,...,1,0,0,0,0,0,0,0,0,0
city_136,0.897,1,1,no_enrollment,0,STEM,18,NaN,NaN,2,...,0,0,0,0,0,0,0,0,0,0
city_100,0.887,1,0,no_enrollment,1,NaN,3,NaN,Pvt Ltd,0,...,0,1,0,0,0,0,0,0,0,0


Выделяем целевую переменную

In [99]:
target = data.education_level
train = data.drop(['education_level','enrolled_university','major_discipline','company_type','company_size','experience','last_new_job'], axis=1) 

In [100]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2129 entries, city_41 to city_102
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city_development_index  2129 non-null   float64
 1   gender                  2129 non-null   int32  
 2   relevent_experience     2129 non-null   int32  
 3   training_hours          2129 non-null   int64  
 4   Full time course        2129 non-null   uint8  
 5   Part time course        2129 non-null   uint8  
 6   no_enrollment           2129 non-null   uint8  
 7   1                       2129 non-null   uint8  
 8   2                       2129 non-null   uint8  
 9   3                       2129 non-null   uint8  
 10  4                       2129 non-null   uint8  
 11  >4                      2129 non-null   uint8  
 12  never                   2129 non-null   uint8  
 13  Arts                    2129 non-null   uint8  
 14  Business Degree         2129 non-nu

Выделяем тренировочную и тестовую выборки

In [117]:
#y - целевая переменнная (target)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size = 0.3, random_state = 42)

N_train, _ = X_train.shape 
N_test,  _ = X_test.shape 
print (N_train, N_test)

1490 639


Построим классификатор логистической регрессии

In [120]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

grid={"C":np.logspace(-5,5,11), "penalty":["l1","l2"]}
logreg=LogisticRegression()
logreg_cv=GridSearchCV(logreg,grid,cv=10)
logreg_cv.fit(X_train,y_train)

GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05]),
                         'penalty': ['l1', 'l2']})

Оценим точность построенного классификатора с помощью метрик precision, recall и F1 на тестовой выборке

In [122]:
from sklearn.model_selection import cross_val_score
print("f1:"+str(np.average(cross_val_score(logreg_cv, X_train, y_train, scoring='f1'))))
print("precision:"+str(np.average(cross_val_score(logreg_cv, X_train, y_train, scoring='precision'))))
print("recall:"+str(np.average(cross_val_score(logreg_cv, X_train, y_train, scoring='recall'))))

f1:0.872065647018388
precision:0.7731543624161075
recall:1.0


Построим классификатор типа Случайный Лес (Random Forest) для решения той же задачи классификации

In [123]:
from sklearn.ensemble import RandomForestClassifier 
param_grid = { 'n_estimators': [100, 150, 200],'max_features': ['auto'],'max_depth' : list(range(1, 15)), 'criterion' :['gini']}
RandForCrit = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_grid, cv = 10, refit = True) 
RandForCrit.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini'],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14],
                         'max_features': ['auto'],
                         'n_estimators': [100, 150, 200]})

Оценим точность построенного классификатора с помощью метрик precision, recall и F1 на тестовой выборке

In [124]:
print("f1:"+str(np.average(cross_val_score(RandForCrit.best_estimator_, X_train, y_train, scoring='f1'))))
print("precision:"+str(np.average(cross_val_score(RandForCrit.best_estimator_, X_train, y_train, scoring='precision'))))
print("recall:"+str(np.average(cross_val_score(RandForCrit.best_estimator_, X_train, y_train, scoring='recall'))))

f1:0.874571481793985
precision:0.7856501417585354
recall:0.9809222661396575


Сравним результаты:

Переменная:        Логистическая регрессия:        Рандомный лес:

f1                 0.872              <            0.874

precision          0.773              <            0.786

recall             1.0                >            0.98

Показатели RandomForest лучше, чем у LogisticRegression